In [1]:
file = './work_dirs/debug_show_deepsort_private/output.pkl'
import mmcv
results = mmcv.load(file)
metric='track'
logger=None
resfile_path=None
bbox_iou_thr=0.5
track_iou_thr=0.5
show_wrong_track=False
out_dir = './work_dirs/visualization'

In [2]:
from mmdet.datasets import build_dataset
from mmcv import Config
from mmtrack.datasets import MOTChallengeDataset
import os
import os.path as osp
import tempfile

import mmcv
import motmetrics as mm
import numpy as np
from mmcv.utils import print_log
from mmdet.core import eval_map
from mmdet.datasets import DATASETS

from mmtrack.core import imshow_wrong_tracks, restore_result
from mmtrack.datasets.coco_video_dataset import CocoVideoDataset

config = cfg = Config.fromfile('configs/mot/deepsort/deepsort_faster-rcnn_fpn_4e_mot17-private-half.py')
dataset = build_dataset(cfg.data.val, dict(test_mode=True))


ImportError: cannot import name 'imshow_wrong_tracks' from 'mmtrack.core' (/home/PJLAB/shensanjing/files/code/mmtracking/mmtrack/core/__init__.py)

In [ ]:
import os
import os.path as osp
import tempfile

import mmcv
import motmetrics as mm
import numpy as np
from mmcv.utils import print_log
from mmdet.core import eval_map
from mmdet.datasets import DATASETS

from mmtrack.core import restore_result
from mmtrack.datasets import CocoVideoDataset

eval_results = dict()
if isinstance(metric, list):
    metrics = metric
elif isinstance(metric, str):
    metrics = [metric]
else:
    raise TypeError('metric must be a list or a str.')
allowed_metrics = ['bbox', 'track']
for metric in metrics:
    if metric not in allowed_metrics:
        raise KeyError(f'metric {metric} is not supported.')

if 'track' in metrics:
    resfiles, names, tmp_dir = dataset.format_results(
        results, resfile_path, metrics)
    print_log('Evaluate CLEAR MOT results.', logger=logger)
    distth = 1 - track_iou_thr

    accs = []
    for name in ['MOT17-05-DPM']:
        print(name)
        if 'half-train' in dataset.ann_file:
            gt_file = osp.join(dataset.img_prefix,
                               f'{name}/gt/gt_half-train.txt')
        elif 'half-val' in dataset.ann_file:
            gt_file = osp.join(dataset.img_prefix,
                               f'{name}/gt/gt_half-val.txt')
        else:
            gt_file = osp.join(dataset.img_prefix, f'{name}/gt/gt.txt')
        res_file = osp.join(resfiles['track'], f'{name}.txt')
        gt = mm.io.loadtxt(gt_file)
        res = mm.io.loadtxt(res_file)
        ini_file = osp.join(dataset.img_prefix, f'{name}/seqinfo.ini')
        if osp.exists(ini_file):
            acc, ana = mm.utils.CLEAR_MOT_M(
                gt, res, ini_file, distth=distth)
        else:
            acc = mm.utils.compare_to_groundtruth(
                gt, res, distth=distth)
        if show_wrong_track or out_dir:
            first_frame_id = acc.mot_events.index[0][0]
            last_frame_id = acc.mot_events.index[-1][0]
            count_fp, count_fn, count_idsw = 0, 0, 0
            for frame_id in range(first_frame_id, last_frame_id + 1):
                # events in the current frame
                print(frame_id)
                events = acc.mot_events.xs(frame_id)
                cur_res = res.loc[frame_id]
                cur_gt = gt.loc[frame_id]
                # path of img
                img = osp.join(dataset.img_prefix, f'{name}/img1/{frame_id:06d}.jpg')
                fps = events[events.Type == 'FP']
                fns = events[events.Type == 'MISS']
                idsws = events[events.Type == 'SWITCH']
                
                bboxes, ids, wrong_types = [], [], []
                for fp_index in fps.index:
                    hid = events.loc[fp_index].HId
                    bboxes.append([cur_res.loc[hid].X, cur_res.loc[hid].Y,
                                   cur_res.loc[hid].X + cur_res.loc[hid].Width,
                                   cur_res.loc[hid].Y + cur_res.loc[hid].Height,
                                   cur_res.loc[hid].Confidence])
                    ids.append(hid)
                    wrong_types.append(0)
                    count_fp += 1
                for fn_index in fns.index:
                    oid = events.loc[fn_index].OId
                    bboxes.append([cur_gt.loc[oid].X, cur_gt.loc[oid].Y,
                                   cur_gt.loc[oid].X + cur_gt.loc[oid].Width,
                                   cur_gt.loc[oid].Y + cur_gt.loc[oid].Height,
                                   cur_gt.loc[oid].Confidence])
                    ids.append(-1)
                    wrong_types.append(1)
                    count_fn += 1
                for idsw_index in idsws.index:
                    hid = events.loc[idsw_index].HId
                    bboxes.append([cur_res.loc[hid].X, cur_res.loc[hid].Y,
                                   cur_res.loc[hid].X + cur_res.loc[hid].Width,
                                   cur_res.loc[hid].Y + cur_res.loc[hid].Height,
                                   cur_res.loc[hid].Confidence])
                    ids.append(-1)
                    wrong_types.append(2)
                    count_idsw += 1
                bboxes = np.asarray(bboxes, dtype=np.float32)
                ids = np.asarray(ids, dtype=np.int32)
                wrong_types = np.asarray(wrong_types, dtype=np.int32)
                img = imshow_wrong_tracks(
                    img,
                    bboxes,
                    ids,
                    wrong_types,
                    show=show_wrong_track,
                    out_file=osp.join(out_dir, f'{name}/{frame_id:06d}.jpg'))
        print(count_fp)
        print(count_fn)
        print(count_idsw)
            
        accs.append(acc)



In [ ]:
    mh = mm.metrics.create()

    summary = mh.compute_many(
        accs,
        names=names,
        metrics=mm.metrics.motchallenge_metrics,
        generate_overall=True)
    str_summary = mm.io.render_summary(
        summary,
        formatters=mh.formatters,
        namemap=mm.io.motchallenge_metric_names)
    print(str_summary)

    eval_results.update({
        mm.io.motchallenge_metric_names[k]: v['OVERALL']
        for k, v in summary.to_dict().items()
    })

    if tmp_dir is not None:
        tmp_dir.cleanup()

if 'bbox' in metrics:
    if isinstance(results, dict):
        bbox_results = results['bbox_results']
    elif isinstance(results, list):
        bbox_results = results
    else:
        raise TypeError('results must be a dict or a list.')
    annotations = [dataset.get_ann_info(info) for info in dataset.data_infos]
    mean_ap, _ = eval_map(
        bbox_results,
        annotations,
        iou_thr=bbox_iou_thr,
        dataset=dataset.CLASSES,
        logger=logger)
    eval_results['mAP'] = mean_ap

for k, v in eval_results.items():
    if isinstance(v, float):
        eval_results[k] = float(f'{(v):.3f}')